In [1]:
import sys
sys.path.append('../')
import lib

In [2]:
import spacy
import json
import pandas as pd

In [3]:
example_path = '../data/articles_ICIBrazza_2020-02-01_2020-04-14.json'
with open(example_path, 'r') as reader:
    data = json.load(reader)
df = pd.DataFrame(columns=data['head'], 
                 data=data['articles'])
print(len(df))
df.head()

108


,id,title,text,Date,country,source,tweet
0,Congo_ICIBrazza_0,Congo : le gouvernement exhorte les parents à ...,L’humanité célèbre le 21 février de chaque ann...,2020-02-23T12:54:50,Congo,ICIBrazza,#Congo : le gouvernement exhorte les parents à...
1,Congo_ICIBrazza_1,Congo : le gouvernement exhorte les parents à ...,L’humanité célèbre le 21 février de chaque ann...,2020-02-23T12:54:50,Congo,ICIBrazza,#Congo : le gouvernement exhorte les parents à...
2,Congo_ICIBrazza_2,Un groupe de banques va racheter une partie de...,Le président Denis Sassou N’Guesso a reçu le 8...,2020-03-11T21:40:54,Congo,ICIBrazza,Un groupe de banques va racheter une partie de...
3,Congo_ICIBrazza_3,Congo : les braqueurs du convoi d’argent arrêt...,Les services départementaux de la police judic...,2020-03-08T11:00:30,Congo,ICIBrazza,#Congo - Les services départementaux de la pol...
4,Congo_ICIBrazza_4,Un groupe de banques va racheter une partie de...,Le président Denis Sassou N’Guesso a reçu le 8...,2020-03-11T21:40:54,Congo,ICIBrazza,Un groupe de banques va racheter une partie de...


In [4]:
#Basic cleaning of the dataframe
clean_df = df.drop('id', axis=1).drop_duplicates(keep='first')

In [5]:
id_ = ['Congo_ICIBrazza_' + str(i) for i in range(len(clean_df))]
index = list(range(len(clean_df)))

In [6]:
clean_df['id'] = id_
clean_df.index = index

## We do not have that much info for a few countries (Congo, RDC, I believe CentrAfique is the same...

# What  do we do with the date ? The format ? Do we split it into year/month/day or sth like that?

In [7]:
#Original text (We have some \n, and potential other codes)
clean_df.iloc[-1].text

'©DR\n\nLe gouvernement congolais a annoncé une enveloppe de quatre milliards de Francs CFA au profit des ménages et personnes pauvres affectés par les mesures de confinement consécutives à la propagation du coronavirus.\n\nCette annonce a été faite ce vendredi par le ministre de la Communication, porte-parole du gouvernement, Thierry Lézin Moungalla à l’issue de la première réunion de la coordination nationale de gestion de la pandémie du Covid-19 qui s’est tenue sous l’autorité du président Sassou N’Guesso\n\nAinsi selon le porte-parole du gouvernement, il ressort de cette réunion plusieurs mesures économiques, sanitaires et sociales.\n\nAu plan social, le gouvernement a décidé du « transfert dans les prochains jours, de la somme de 4 milliards de francs CFA aux ménages et personnes pauvres déjà répertoriés dans différentes localités du pays par le ministère en charge des affaires sociales. »\n\nLa Coordination a réitéré la gratuité de l’eau et de l’électricité pour tous les ménages,

In [8]:
#Download spacy french vocabulary 
#!python -m spacy download fr_core_news_md
#They also have multi-lingual stuff--> Can be interesting if we want to apply the method to other types of text

In [9]:
try:
    spacy.load('fr_core_news_md')
except Exception as e:
    print(e)
#That's the usual way to make it work... I don't know why it doesn't here
    

In [11]:
import fr_core_news_md
nlp = fr_core_news_md.load()

In [13]:
def clean_text(text_):
    text_ = text_.rstrip() 
    text_ = text_.replace('\n', ' ')
    text_ = text_.replace("  ", " ")
    text_ = text_.replace('[^\w\s]','')
    #lower_ = lambda x: " ".join(x.lower() for x in x.split())
    #text = lower_(text)
    doc = nlp(text_)
    return doc
    #get the "entities in the text"

Text = clean_df.iloc[-1].text
doc = clean_text(Text)
print(doc.ents)

(DR, Francs, ministre de la Communication, Thierry Lézin Moungalla, Covid-19, Sassou N’Guesso, CFA, La Coordination, Coordination nationale, Task, Covid-19, Trésor, Etat, Covid-19, Covid-19, Covid-19, Covid-19)


In [14]:
help(doc)

Help on Doc object:

class Doc(builtins.object)
 |  A sequence of Token objects. Access sentences and named entities, export
 |  annotations to numpy arrays, losslessly serialize to compressed binary
 |  strings. The `Doc` object holds an array of `TokenC` structs. The
 |  Python-level `Token` and `Span` objects are views of this array, i.e.
 |  they don't own the data themselves.
 |  
 |  EXAMPLE: Construction 1
 |      >>> doc = nlp(u'Some text')
 |  
 |      Construction 2
 |      >>> from spacy.tokens import Doc
 |      >>> doc = Doc(nlp.vocab, words=[u'hello', u'world', u'!'],
 |                    spaces=[True, False, False])
 |  
 |  DOCS: https://spacy.io/api/doc
 |  
 |  Methods defined here:
 |  
 |  __bytes__(...)
 |  
 |  __getitem__(...)
 |      Get a `Token` or `Span` object.
 |      
 |      i (int or tuple) The index of the token, or the slice of the document
 |          to get.
 |      RETURNS (Token or Span): The token at `doc[i]]`, or the span at
 |          `doc[sta

In [25]:
#print([t.text for t in doc]) 
#doc.to_json()
#for i, sent in enumerate(doc.sents):
#    print(i, sent)

In [57]:
from collections import Counter
count = Counter()
count.update(map(lambda x : x.text, doc.ents))
count

Counter({'DR': 1,
         'Francs': 1,
         'ministre de la Communication': 1,
         'Thierry Lézin Moungalla': 1,
         'Covid-19': 6,
         'Sassou N’Guesso': 1,
         'CFA': 1,
         'La Coordination': 1,
         'Coordination nationale': 1,
         'Task': 1,
         'Trésor': 1,
         'Etat': 1})

# Things that could be interesting for the very first analysis:
- Count of tweets with time (probably more tweets/news since mid-march)
- Keywords with time


In [85]:
#choose index:
import datetime
index = 20
date_str = clean_df.iloc[index].Date.split('T')[0]

In [87]:
#Timestamp.valueOf("2018-11-12 01:02:03.123456789")
element

datetime.datetime(2020, 2, 17, 0, 0)

In [84]:
clean_df.iloc[index].Date.split('T')[1]

'22:32:24'

In [177]:
key_words = ['Corona', 'Covid19', 'Covid', 'Covid-19', 'Virus', 'Pandémie', 'Maladie', 'Coronavirus']
low_key_words = map(lambda x: x.lower(), key_words)
key_words.extend(list(low_key_words))
key_words


['Corona',
 'Covid19',
 'Covid',
 'Covid-19',
 'Virus',
 'Pandémie',
 'Maladie',
 'Coronavirus',
 'corona',
 'covid19',
 'covid',
 'covid-19',
 'virus',
 'pandémie',
 'maladie',
 'coronavirus']

In [205]:
import datetime
import seaborn as sns 
sns.set_style("whitegrid")

def clean_date(date_str):
    """
    Modify the date into a timestamp format
    """
    dmy, hms = date_str.split('T')
    dmy = "/".join(dmy.split('-')[::-1])
    element = datetime.datetime.strptime(dmy,
                                         "%d/%m/%Y") 
    return element
    
#clean_df['Date'] = clean_df['Date'].apply(clean_date)


AttributeError: 'Timestamp' object has no attribute 'split'

In [209]:
clean_df = clean_df.sort_values('Date')
clean_df
clean_df.reset_index(drop=True, inplace=True)
clean_df

,title,text,Date,country,source,tweet,id
0,Hémicycles d'Afrique,"Parti de Chine, le Covid-19 s’est répandu à tr...",2020-02-02,Congo,ICIBrazza,Le tout premier numéro du magazine online de \...,Congo_ICIBrazza_5
1,Congo : travail à minima au ministère des fina...,Indisponible depuis quelques mois pour des rai...,2020-02-07,Congo,ICIBrazza,#Congo : travail à minima au ministère des fin...,Congo_ICIBrazza_17
2,Congo – Santé : Le gouvernement alloue 200 mil...,Le gouvernement congolais a acheminé une somme...,2020-02-07,Congo,ICIBrazza,#Congo - #Santé : Le gouvernement alloue 200 m...,Congo_ICIBrazza_16
3,Judo : Teddy Riner éliminé après 154 victoires...,"Après 154 victoires consécutives en carrière, ...",2020-02-09,Congo,ICIBrazza,#JUDO - #TeddyRiner s’est incliné dimanche au ...,Congo_ICIBrazza_3
4,Bissau : l’ONU ne reconnaît pas la victoire de...,La déclaration du Secrétaire général de l’Orga...,2020-02-09,Congo,ICIBrazza,#Bissau : l'ONU ne reconnaît pas la victoire d...,Congo_ICIBrazza_4
5,Congo – Fnuap : Mabingué Ngom s’engage à inves...,Le directeur régional du Fonds des nations uni...,2020-02-12,Congo,ICIBrazza,"#Congo Le directeur régional du Fnuap, @Mabin...",Congo_ICIBrazza_27
6,Congo : les ailes de poulet reviennent sur les...,Quasi absents des tables de différents marchés...,2020-02-13,Congo,ICIBrazza,#Congo : les ailes de poulet reviennent sur le...,Congo_ICIBrazza_26
7,"Pour « penser le Congo de demain », Marien Fau...",Il en décline la vision dans cet entretien et ...,2020-02-14,Congo,ICIBrazza,"Pour « penser le Congo de demain », Marien Fa...",Congo_ICIBrazza_25
8,France: deux hommes écroués après une attaque ...,Deux hommes ont été inculpés et écroués après ...,2020-02-17,Congo,ICIBrazza,#France: deux hommes écroués après une attaque...,Congo_ICIBrazza_22
9,Congo : travail à minima au ministère des fina...,Indisponible depuis quelques mois pour des rai...,2020-02-17,Congo,ICIBrazza,#Congo : travail à minima au ministère des fin...,Congo_ICIBrazza_21


In [332]:
#clean_df.head()
#for index in range(len(clean_df)):
%matplotlib notebook
from ipywidgets import interact

import plotly.graph_objects as go

@interact(index=(0,len(clean_df)))
def visualize(index):
    df = clean_df
    count = Counter()
    Text = df.iloc[index].text
    Title = df.iloc[index].title
    doc = clean_text(Title+Text)
    key_words = ['Corona', 'Covid19', 
                 'Covid', 'Covid-19', 
                 'Virus', 'Pandémie', 
                 'Maladie', 'Coronavirus', 
                'Santé']
    low_key_words = map(lambda x: x.lower(), key_words)
    key_words.extend(list(low_key_words))
    #print(Text)
    print(Title, df.iloc[index].Date)
    count = Counter()
    count_key = Counter()
    for token in doc:
        if token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']:
            #print(token.text, token.pos_, token.dep_)
            count.update([token.text.lower()])
        if token.text in key_words:
            count_key.update([token.text.lower()])
    print(count_key)
    #count.most_common()[:10]
    #doc = clean_text(Text) #Get the doc
    #count.update(map(lambda x : x.text, doc.ents))
    #count.update(#Text+Title)
    #===============
    # Plot
    #plt.figure(figsize=(10,5))
    #fig, ax = plt.subplots(2,1,figsize=(10,5))
    labels, values = zip(*count.most_common()[:10])
    fig = go.Figure(data=[go.Histogram(x=labels, y=values, histfunc='sum')])
    fig.show()

    #indexes = np.arange(len(labels))
    #width = 0.5

    #ax[0].bar(indexes, values, width, alpha=0.6, color='grey')
    #ax[0].set_xticks(indexes + width * 0.01, labels)
    #ax[0].axis.set_xticks(indexes + width * 0.01, labels)
    #ax[0].set_xticklabels(labels)
    try:
        labels_key, values_key = zip(*count_key.most_common()[:10])
        #indexes = np.arange(len(labels_key))
        #width = 0.5
    
        #ax[1].bar(indexes, values_key, width, alpha=0.5, color='red')
        #ax[1].set_xticks(indexes + width * 0.01, labels_key)
        #ax[1].set_xticks(indexes + width * 0.01, labels_key)
        #ax[1].set_xticklabels(labels_key)
        #print(labels_key, values_key)
        fig = go.Figure(data=[go.Histogram(x=labels_key, y=values_key, histfunc='sum')])
        fig.show()
    except:
        print('No "key word" in this doc')
    #return fig,ax
    
#@interact(index=(0,70)    
#visualize(40)

interactive(children=(IntSlider(value=39, description='index', max=78), Output()), _dom_classes=('widget-inter…

In [100]:
"""from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
# use interact decorator to decorate the function, so the function can receive the slide bar's value with parameter x.
@interact(x=(0, 100))
def double_number(x):
    print("The double of %d is %d." % (x, x*2))
    X = np.linspace(0,x)
    y = np.cos(X)
    plt.plot(X,y)
    plt.show()
"""

interactive(children=(IntSlider(value=50, description='x'), Output()), _dom_classes=('widget-interact',))

In [284]:
#
def get_stats(index):
    df = clean_df
    count = Counter()
    Text = df.iloc[index].text
    Title = df.iloc[index].title
    date = df.iloc[index].Date
    doc = clean_text(Title+Text)
    key_words = ['Corona', 'Covid19', 
                 'Covid', 'Covid-19', 
                 'Virus', 'Pandémie', 
                 'Maladie', 'Coronavirus', 
                'Santé']
    low_key_words = map(lambda x: x.lower(), key_words)
    key_words.extend(list(low_key_words))
    print(Title, df.iloc[index].Date)
    count = Counter()
    count_key = Counter()
    for token in doc:
        if token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']:
            #print(token.text, token.pos_, token.dep_)
            count.update([token.text.lower()])
        if token.text in key_words:
            count_key.update([token.text.lower()])
    return Title, count_key, date

freq_ = []
for index in range(len(clean_df)):
    _, counter, date = get_stats(index)
    freq_.append(sum(counter.values()))


    

Hémicycles d'Afrique 2020-02-02 00:00:00
Congo : travail à minima au ministère des finances qui pourra remplacer Calixte Ganongo ? 2020-02-07 00:00:00
Congo – Santé : Le gouvernement alloue 200 millions de Fcfa pour soutenir les congolais présents à Wuhan 2020-02-07 00:00:00
Judo : Teddy Riner éliminé après 154 victoires de suite 2020-02-09 00:00:00
Bissau : l’ONU ne reconnaît pas la victoire de Umaro Embalo à la présidentielle 2020-02-09 00:00:00
Congo – Fnuap : Mabingué Ngom s’engage à investir sur la question des jeunes et des femmes 2020-02-12 00:00:00
Congo : les ailes de poulet reviennent sur les marchés 2020-02-13 00:00:00
Pour « penser le Congo de demain », Marien Fauney Ngombé lance les ACC (Ateliers Citoyens du Congo) en ce début d’année 2020. 2020-02-14 00:00:00
France: deux hommes écroués après une attaque à main armée contre un diplomate congolais 2020-02-17 00:00:00
Congo : travail à minima au ministère des finances qui pourra remplacer Calixte Ganongo ? 2020-02-17 00:00:

NameError: name 'df_clean' is not defined

In [313]:
clean_df['freq'] = freq_
import plotly.express as px
#df = px.data.tips()
fig = px.histogram(clean_df, x="Date", y="freq", marginal="rug",  histfunc='sum', nbins=20)#box, rug

fig.show()

In [153]:
#doc.count_by(7)
#help(doc)
count = Counter()
for token in doc:
    if token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']:
        print(token.text, token.pos_, token.dep_)
        count.update([token.text])
count.most_common()[:10]

DR VERB ROOT
gouvernement NOUN nsubj
congolais ADJ amod
annoncé VERB ROOT
enveloppe NOUN obj
milliards NOUN nmod
Francs PROPN nmod
CFA PROPN flat:name
profit NOUN case
ménages NOUN nmod
personnes NOUN conj
pauvres ADJ amod
affectés VERB acl
mesures NOUN obl
confinement NOUN nmod
consécutives NOUN amod
propagation NOUN obl
coronavirus NOUN nmod
annonce NOUN nsubj:pass
faite VERB ROOT
vendredi NOUN obl
ministre NOUN obl
Communication NOUN nmod
porte-parole NOUN appos
gouvernement NOUN nmod
Thierry PROPN dep
Lézin PROPN flat:name
Moungalla PROPN flat:name
issue NOUN fixed
première ADJ amod
réunion NOUN obl
coordination NOUN nmod
nationale ADJ amod
gestion NOUN nmod
pandémie NOUN nmod
Covid-19 NOUN nmod
tenue VERB acl:relcl
autorité NOUN obl
président NOUN nmod
Sassou PROPN flat:name
N’ VERB flat:name
Guesso PROPN flat:name
porte-parole NOUN obl
gouvernement NOUN nmod
ressort NOUN advmod
réunion NOUN nmod
mesures NOUN nmod
économiques ADJ amod
sanitaires ADJ conj
sociales NOUN conj
plan NO

[('Covid-19', 6),
 ('tous', 5),
 ('gouvernement', 4),
 ('santé', 4),
 ('ménages', 3),
 ('plan', 3),
 ('congolais', 2),
 ('milliards', 2),
 ('CFA', 2),
 ('personnes', 2),
 ('pauvres', 2),
 ('mesures', 2),
 ('confinement', 2),
 ('propagation', 2),
 ('porte-parole', 2),
 ('réunion', 2),
 ('nationale', 2),
 ('pandémie', 2),
 ('économiques', 2),
 ('sociales', 2),
 ('jours', 2),
 ('aux', 2),
 ('pays', 2),
 ('charge', 2),
 ('Coordination', 2),
 ('économique', 2),
 ('lutte', 2),
 ('s’', 2),
 ('DR', 1),
 ('annoncé', 1),
 ('enveloppe', 1),
 ('Francs', 1),
 ('profit', 1),
 ('affectés', 1),
 ('consécutives', 1),
 ('coronavirus', 1),
 ('annonce', 1),
 ('faite', 1),
 ('vendredi', 1),
 ('ministre', 1),
 ('Communication', 1),
 ('Thierry', 1),
 ('Lézin', 1),
 ('Moungalla', 1),
 ('issue', 1),
 ('première', 1),
 ('coordination', 1),
 ('gestion', 1),
 ('tenue', 1),
 ('autorité', 1),
 ('président', 1),
 ('Sassou', 1),
 ('N’', 1),
 ('Guesso', 1),
 ('ressort', 1),
 ('sanitaires', 1),
 ('social', 1),
 ('décid

In [270]:
"""
import matplotlib.animation as animation
def animate_hist():
    fig = plt.figure()
    ims = []
    for i in range(2):
        #Show the geopotential--> 0
        #Show altitude -->1
        #data = vision_data[n, i, 0, 1]
        #img = plt.imshow(data, animated=True)
        fig, ax = visualize(i)
        img = ax
        ims.append([img])
        #plt.show()

    ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True,
                                    repeat_delay=1000)
    return ani
"""

In [294]:
#animate_hist()